In [1]:
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd
import calendar

In [17]:
list_of_loads = [
    "P_CabinePower_L1",
    "P L1 WS Emergency Light",
    "P L1 WS Light",
    "P L1 WS FanCoil",
    "P L1 WS FM",
    "P_HeatPump",
    "P L2 WS Emergency Light",
    "P L2 WS Light",
    "P L2 WS FanCoil",
    "P L2 WS FM",
    "P_CabineLight_L3",
    "P L3 WS Emergency Light",
    "P L3 WS Light",
    "P L3 WS FanCoil",
    "P L3 WS FM"
]

In [3]:
min_power = {
    "P_CabinePower_L1": 50,
    "P L1 WS Emergency Light": 150,
    "P L1 WS Light": 200,
    "P L1 WS FanCoil": 0,
    "P L1 WS FM": 0,
    "P_HeatPump": 0,
    "P L2 WS Emergency Light": 200,
    "P L2 WS Light": 200,
    "P L2 WS FanCoil": 0,
    "P L2 WS FM": 0,
    "P_CabineLight_L3": 0,
    "P L3 WS Emergency Light": 0,
    "P L3 WS Light": 0,
    "P L3 WS FanCoil": 0,
    "P L3 WS FM": 200
}

In [4]:
max_power = {
    "P_CabinePower_L1": 150,
    "P L1 WS Emergency Light": 1500,
    "P L1 WS Light": 350,
    "P L1 WS FanCoil": 200,
    "P L1 WS FM": 4000,
    "P_HeatPump": 1000,
    "P L2 WS Emergency Light": 1700,
    "P L2 WS Light": 400,
    "P L2 WS FanCoil": 200,
    "P L2 WS FM": 1300,
    "P_CabineLight_L3": 10,
    "P L3 WS Emergency Light": 1000,
    "P L3 WS Light": 350,
    "P L3 WS FanCoil": 200,
    "P L3 WS FM": 2000
}

In [5]:
dataset_path = os.path.join("Preprocessing", "Dataset", "df.csv")
df = pd.read_csv(dataset_path, index_col="Time")
df.index = pd.to_datetime(df.index)

In [6]:
for load in list_of_loads:
    df[load] = df[load].where(
        (df[load] >= min_power[load]) & (df[load] <= max_power[load]), np.nan
    )

In [9]:
df_avg = df.resample("H").mean()

In [12]:
df_weekday_avg = df_avg[df_avg.index.weekday < 5]
df_weekend_avg = df_avg[df_avg.index.weekday >= 5]

In [13]:
df_tmp_weekday = df[df.index.weekday < 5]
df_tmp_weekend = df[df.index.weekday >= 5]

In [14]:
df_weekday_avg_hour = df_tmp_weekday.groupby(df_tmp_weekday.index.hour).mean()
df_weekend_avg_hour = df_tmp_weekend.groupby(df_tmp_weekend.index.hour).mean()

In [20]:
def fill_empty_value(df, df_tmp):
    for index, row in df.iterrows():
        for load in list_of_loads:
            if pd.isna(row[load]):
                df.loc[index][load] = df_tmp.loc[index.hour][load]
    return df

In [21]:
df_weekday_avg = fill_empty_value(df_weekday_avg, df_weekday_avg_hour)
df_weekend_avg = fill_empty_value(df_weekend_avg, df_weekend_avg_hour)

In [ ]:
df_weekend_avg.describe()